In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("kilian-group/arxiv-classifier", "default")

c:\Users\Iskandar\Documents\GitHub\Tubes-NLP-PaperMatch\hf_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Convert to pandas
train_df = ds["train"].to_pandas()
test_df  = ds["test"].to_pandas()

In [3]:
train_df = train_df[["field", "abstract"]]
test_df  = test_df[["field", "abstract"]]

In [4]:
train_df.head()

,field,abstract
0,cond-mat,An electric current controlled spin-wave logic...
1,cond-mat,We investigate nanoelectromechanical systems n...
2,cond-mat,We have investigated the polarization dependen...
3,cond-mat,The erasure of a classical bit of information ...
4,cond-mat,While mesoscopic conducting loops are sensitiv...


In [5]:
test_df.head()

,field,abstract
0,cond-mat,Proximity-induced magnetic effects on the surf...
1,cond-mat,We present the first experimental microwave re...
2,cond-mat,We report on the effect of the lateral confine...
3,cond-mat,Measurement of gravitational Hawking radiation...
4,cond-mat,We study the non-equilibrium evolution of conc...


In [6]:
import pandas as pd
import nltk

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

In [8]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Iskandar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Iskandar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Iskandar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Iskandar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [10]:
def preprocess(text):
  text = str(text).lower()

  tokens = word_tokenize(text)

  tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

  return " ".join(tokens)

In [11]:
train_df["new_abstract"] = train_df["abstract"].apply(preprocess)

In [12]:
test_df["new_abstract"] = test_df["abstract"].apply(preprocess)

In [13]:
from gensim.models import KeyedVectors

ft_pretrained = KeyedVectors.load_word2vec_format('cc.en.300.vec')
vector = ft_pretrained['friend']

In [14]:
import re

texts = []
for i in range(len(train_df)):
    text = re.sub('[^a-zA-Z]', ' ', train_df.iloc[i]['new_abstract'])
    text = text.lower()
    text = text.split()
    text = ' '.join(text)
    texts.append(text)


In [15]:
sentences = [line.split() for line in texts]

In [16]:
from gensim.models import Word2Vec

In [17]:
w2v = Word2Vec(sentences, vector_size=100, window=5, workers=4, epochs=10, min_count=5)

In [18]:
print(sentences[20:25])

[['topological', 'metal', 'continue', 'attract', 'attention', 'novel', 'gapless', 'state', 'matter', 'exists', 'exhaustive', 'classification', 'possible', 'topologically', 'nontrivial', 'metallic', 'state', 'observable', 'property', 'follow', 'electronic', 'structure', 'topology', 'less', 'well', 'understood', 'present', 'study', 'electromagnetic', 'response', 'three', 'dimensional', 'topological', 'metal', 'weyl', 'dirac', 'node', 'spectrum', 'systematizes', 'extends', 'earlier', 'pioneering', 'study', 'particular', 'argue', 'smoking', 'gun', 'feature', 'chiral', 'anomaly', 'topological', 'metal', 'existence', 'propagating', 'chiral', 'density', 'mode', 'even', 'regime', 'weak', 'magnetic', 'field', 'also', 'demonstrate', 'optical', 'conductivity', 'metal', 'exhibit', 'extra', 'peak', 'exists', 'top', 'standard', 'metallic', 'drude', 'peak', 'spectral', 'weight', 'peak', 'transferred', 'high', 'frequency', 'width', 'proportional', 'chiral', 'charge', 'relaxation', 'rate'], ['dirac', '

In [19]:
from gensim.models import FastText
from gensim.test.utils import common_texts

ft = FastText(sentences=sentences, vector_size=100, window=5, min_count=5, workers=4, sg=1)

similar_words = ft.wv.most_similar('friend')

print("Most similar words to 'friend':", similar_words)

Most similar words to 'friend': [('friends', 0.9271597266197205), ('friendship', 0.874294638633728), ('friendlier', 0.7858349680900574), ('friendly', 0.751064658164978), ('friendliness', 0.7322483062744141), ('politician', 0.7315405607223511), ('unfriendly', 0.7187352180480957), ('people', 0.7116178274154663), ('socialist', 0.7012622356414795), ('engages', 0.6981586217880249)]


In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

print("Number of unique words in dictionary =", len(tokenizer.word_index))
print("Dictionary is =", tokenizer.word_index)

Number of unique words in dictionary = 113844
Dictionary is = {'model': 1, 'system': 2, 'result': 3, 'method': 4, 'data': 5, 'show': 6, 'time': 7, 'based': 8, 'two': 9, 'using': 10, 'n': 11, 'paper': 12, 'problem': 13, 'also': 14, 'network': 15, 'study': 16, 'one': 17, 's': 18, 'algorithm': 19, 'state': 20, 'function': 21, 'approach': 22, 'field': 23, 'present': 24, 'new': 25, 'number': 26, 'space': 27, 'different': 28, 'k': 29, 'used': 30, 'set': 31, 'structure': 32, 'x': 33, 'energy': 34, 'case': 35, 'order': 36, 'analysis': 37, 'non': 38, 'e': 39, 'g': 40, 'well': 41, 'high': 42, 'theory': 43, 'property': 44, 'p': 45, 'first': 46, 'process': 47, 'parameter': 48, 'proposed': 49, 'large': 50, 'solution': 51, 'equation': 52, 'dynamic': 53, 'c': 54, 'work': 55, 'use': 56, 'distribution': 57, 'application': 58, 'group': 59, 'd': 60, 'type': 61, 'information': 62, 'performance': 63, 'point': 64, 'r': 65, 'simulation': 66, 'graph': 67, 'effect': 68, 'term': 69, 'phase': 70, 'value': 71, 'c

In [22]:
def embedding_for_vocab(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # +1 for padding token (index 0)
    embedding_matrix_vocab = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix_vocab

In [23]:
embedding_dim = 100
glove_path = './glove.6B.100d.txt'

embedding_matrix_vocab = embedding_for_vocab(glove_path, tokenizer.word_index, embedding_dim)

In [24]:
first_word_index = 1
print("Dense vector for word with index 1 =>", embedding_matrix_vocab[first_word_index])

Dense vector for word with index 1 => [-0.27663001  0.55093998  0.13618     0.59683001 -0.24868    -0.34311
  0.74659997  0.12718    -0.28819001  0.78735    -0.020772   -0.48315999
  0.23378    -0.0064957   0.33921999  0.042282    0.54477     0.60575002
  0.27294001  0.13011    -0.0439     -0.51363999  0.25549999  0.33046001
  0.43983001 -0.22454999  0.092492   -1.23899996 -0.58248001  0.29795
  0.12634     1.31089997 -0.22366001 -0.18946999  0.83993     0.38651001
  0.41031    -0.047329    0.33790001 -0.43654999 -0.19865    -0.30695999
  0.12327     0.22901    -0.34215999  0.1925      0.15737    -0.3082
 -0.39695999  0.080463   -0.28005999  0.081266    0.15053999  1.38429999
  0.39917001 -1.99699998 -0.74010998  0.59262002  1.09000003 -0.11469
  0.092272    0.33024001 -0.61947     0.63913     1.18570006 -0.042564
 -0.60857999 -0.30320001  0.16855     0.56585997  0.36660999  0.12138
 -0.060965   -0.057785    0.47064    -0.1027      1.1243     -0.026558
 -0.81186002 -0.94243997 -0.09335

In [25]:
from sklearn.model_selection import train_test_split

X_train_text, X_val_text, y_train, y_val = train_test_split(
    train_df["new_abstract"],
    train_df["field"],
    test_size=0.2,
    random_state=42,
    stratify=train_df["field"]
)

In [26]:
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, SimpleRNN, Dense, Dropout, Bidirectional, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

max_words = 20000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_df['new_abstract'])

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_val_seq = tokenizer.texts_to_sequences(X_val_text)

max_len = 200

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)
num_classes = len(le.classes_)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_enc),
    y=y_train_enc
)
class_weights = dict(enumerate(class_weights))

def create_embedding_matrix_ft_pretrained(word_index, embedding_model, embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word_index.items():
        if word in embedding_model:
            embedding_matrix[idx] = embedding_model[word]
    return embedding_matrix

def create_embedding_matrix(word_index, embedding_model, embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word_index.items():
        if word in embedding_model.wv:
            embedding_matrix[idx] = embedding_model.wv[word]
    return embedding_matrix

w2v_matrix = create_embedding_matrix(tokenizer.word_index, w2v, 100)
ft_matrix = create_embedding_matrix(tokenizer.word_index, ft, 100)
ft_pretrained_matrix = create_embedding_matrix_ft_pretrained(tokenizer.word_index, ft_pretrained, 300)
glove_matrix = embedding_matrix_vocab

def build_cnn(embedding_matrix, max_len, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_rnn(embedding_matrix, max_len, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))
    model.add(Bidirectional(SimpleRNN(128, return_sequences=False)))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_lstm(embedding_matrix, max_len, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_ann(embedding_matrix, max_len, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model

embedding_dict = {
    "ft": ft_matrix,
    "ft_pretrained": ft_pretrained_matrix,
    "w2v_matrix": w2v_matrix,
    "glove": glove_matrix
}

model_dict = {
    "CNN": build_cnn,
    "RNN": build_rnn,
    "LSTM": build_lstm,
    "ANN": build_ann
}

results = {}

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

for emb_name, emb_matrix in embedding_dict.items():
    for model_name, model_func in model_dict.items():
        print(f"Training {model_name} with {emb_name} embedding...")
        model = model_func(emb_matrix, max_len, num_classes)
        model.fit(X_train_pad, y_train_enc,
                  epochs=30, batch_size=64,
                  validation_data=(X_val_pad, y_val_enc),
                  class_weight=class_weights,
                  callbacks=[early_stop])
        
        model_path = f"models/{model_name}_{emb_name}.h5"
        model.save(model_path)
        print(f"Model saved to {model_path}")

        y_pred_prob = model.predict(X_val_pad)
        y_pred = np.argmax(y_pred_prob, axis=1)

        acc = accuracy_score(y_val_enc, y_pred)
        report = classification_report(y_val_enc, y_pred, target_names=le.classes_, zero_division=0)

        results[f"{model_name}_{emb_name}"] = {"accuracy": acc, "report": report}
        print(f"Accuracy for {model_name}_{emb_name}: {acc}")
        print(report)

Training CNN with ft embedding...


c:\Users\Iskandar\Documents\GitHub\Tubes-NLP-PaperMatch\hf_env\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 90s 64ms/step - accuracy: 0.4435 - loss: 1.6286 - val_accuracy: 0.5631 - val_loss: 1.2590
Epoch 2/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 85s 62ms/step - accuracy: 0.5660 - loss: 1.1648 - val_accuracy: 0.6757 - val_loss: 1.0259
Epoch 3/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 84s 62ms/step - accuracy: 0.6004 - loss: 1.0475 - val_accuracy: 0.5665 - val_loss: 1.2453
Epoch 4/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 83s 61ms/step - accuracy: 0.6137 - loss: 0.9580 - val_accuracy: 0.6178 - val_loss: 1.1599
Epoch 5/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 83s 61ms/step - accuracy: 0.6272 - loss: 0.8855 - val_accuracy: 0.6268 - val_loss: 1.0772
Epoch 6/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 83s 61ms/step - accuracy: 0.6419 - loss: 0.8088 - val_accuracy: 0.6281 - val_loss: 1.0560
Epoch 7/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 82s 60ms/step - accuracy: 0.6481 - loss: 0.7566 - val_accuracy: 0.5994 - val_loss: 1.1588


Model saved to models/CNN_ft.h5
680/680 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step
Accuracy for CNN_ft: 0.6756669733210672
              precision    recall  f1-score   support

    astro-ph       0.74      0.84      0.79      1117
    cond-mat       0.61      0.71      0.65      1439
          cs       0.80      0.76      0.78      5578
        econ       0.14      0.47      0.22        86
        eess       0.33      0.48      0.39       599
       gr-qc       0.33      0.71      0.45       160
      hep-ex       0.42      0.84      0.56       160
     hep-lat       0.62      0.91      0.74       160
      hep-ph       0.54      0.49      0.51       160
      hep-th       0.39      0.65      0.49       160
        math       0.88      0.82      0.85      4639
     math-ph       0.14      0.34      0.20       160
        nlin       0.37      0.73      0.49       700
     nucl-ex       0.41      0.49      0.45       160
     nucl-th       0.46      0.61      0.53       160
     physics       0.7

c:\Users\Iskandar\Documents\GitHub\Tubes-NLP-PaperMatch\hf_env\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


...

Epoch 1/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1366s 1s/step - accuracy: 0.0821 - loss: 2.8990 - val_accuracy: 0.0814 - val_loss: 2.5815
Epoch 2/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1335s 982ms/step - accuracy: 0.1369 - loss: 2.3238 - val_accuracy: 0.2028 - val_loss: 2.2913
Epoch 3/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1276s 939ms/step - accuracy: 0.2589 - loss: 1.8902 - val_accuracy: 0.3141 - val_loss: 1.9571
Epoch 4/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1324s 974ms/step - accuracy: 0.3297 - loss: 1.6111 - val_accuracy: 0.3669 - val_loss: 1.6976
Epoch 5/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1283s 944ms/step - accuracy: 0.3824 - loss: 1.4011 - val_accuracy: 0.4451 - val_loss: 1.5103


Model saved to models/LSTM_glove.h5
680/680 ━━━━━━━━━━━━━━━━━━━━ 137s 200ms/step
Accuracy for LSTM_glove: 0.08141674333026679
              precision    recall  f1-score   support

    astro-ph       0.36      0.26      0.30      1117
    cond-mat       0.09      0.01      0.02      1439
          cs       0.37      0.04      0.07      5578
        econ       0.00      0.03      0.01        86
        eess       0.06      0.46      0.11       599
       gr-qc       0.00      0.00      0.00       160
      hep-ex       0.12      0.39      0.19       160
     hep-lat       0.05      0.26      0.08       160
      hep-ph       0.07      0.19      0.11       160
      hep-th       0.03      0.05      0.04       160
        math       0.43      0.02      0.04      4639
     math-ph       0.02      0.27      0.03       160
        nlin       0.05      0.15      0.07       700
     nucl-ex       0.09      0.14      0.11       160
     nucl-th       0.02      0.02      0.02       160
     phys

c:\Users\Iskandar\Documents\GitHub\Tubes-NLP-PaperMatch\hf_env\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 76s 55ms/step - accuracy: 0.0931 - loss: 2.9868 - val_accuracy: 0.1080 - val_loss: 2.9141
Epoch 2/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 74s 55ms/step - accuracy: 0.0800 - loss: 2.9340 - val_accuracy: 0.1386 - val_loss: 2.8565
Epoch 3/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 74s 54ms/step - accuracy: 0.0777 - loss: 2.8796 - val_accuracy: 0.1238 - val_loss: 2.8319
Epoch 4/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 74s 54ms/step - accuracy: 0.0911 - loss: 2.8237 - val_accuracy: 0.1571 - val_loss: 2.7734
Epoch 5/30
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 74s 55ms/step - accuracy: 0.1077 - loss: 2.7619 - val_accuracy: 0.1612 - val_loss: 2.7263


Model saved to models/ANN_glove.h5
680/680 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step
Accuracy for ANN_glove: 0.10800367985280589
              precision    recall  f1-score   support

    astro-ph       0.12      0.58      0.20      1117
    cond-mat       0.08      0.02      0.03      1439
          cs       0.00      0.00      0.00      5578
        econ       0.00      0.00      0.00        86
        eess       0.04      0.35      0.07       599
       gr-qc       0.00      0.00      0.00       160
      hep-ex       0.00      0.00      0.00       160
     hep-lat       0.01      0.07      0.01       160
      hep-ph       0.00      0.00      0.00       160
      hep-th       0.00      0.00      0.00       160
        math       0.62      0.25      0.36      4639
     math-ph       0.02      0.30      0.03       160
        nlin       0.04      0.21      0.07       700
     nucl-ex       0.00      0.00      0.00       160
     nucl-th       0.00      0.00      0.00       160
     physics   

In [ ]:
import os
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

X_test_seq = tokenizer.texts_to_sequences(test_df['new_abstract'])
X_test_pad = pad_sequences(X_test_seq, maxlen=200)

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(test_df['field'])

print("Testing saved models...")
for model_file in os.listdir("models"):
    if model_file.endswith(".h5"):
        model_path = os.path.join("models", model_file)
        print(f"Loading {model_file} ...")
        loaded_model = load_model(model_path)
        
        y_pred_prob = loaded_model.predict(X_test_pad)
        y_pred = np.argmax(y_pred_prob, axis=1)
        
        acc = accuracy_score(y_test_enc, y_pred)
        report = classification_report(y_test_enc, y_pred, target_names=le.classes_, zero_division=0)
        print(f"Loaded model {model_file} accuracy: {acc}")
        print(report)
        print("="*50)